# --> Importations

In [50]:
import tensorflow as tf
import numpy as np

# --> Custom layers class

In [55]:
#Jusqu'ici on utilisait Dense() pour creer notre layer, Dense() permet de creer une couche
#qui est connecte avec tous les neurones de la couche precedente. On va voir ici comment
#creer notre propre couches. Conv() est un autre type de couche disponible avec tensorflow.
#On va ici creer notre propre classe grace au subclassing.
class CustomLayer(tf.keras.layers.Layer): #Ici on va creer une MLP layer
    
    def __init__(self, units, activations, **kwargs): #Initialisation des layers
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units #Nombre de layers que l'on doit creer
        self.activations = activations
        self.weights_list = [] #Initialisation de notre liste de poids dans la couche
        
    #Appel lors du premier predict(), permet de creer tous les poids de nos couches
    def build(self, input_shape):
        layerNb = 0;
        for unit in self.units:
            print("Unit : ", unit, ", Input_shape : ", input_shape)
            #On utilise la fonction add_weight vu qu'on herite de la classe keras Layer
            weights = self.add_weight(              
                            name="weight-%s" % (layerNb), #Nom des poids
                            #Shape des poids, input_shape = (None, 10) et unit = [4, 2]
                            shape=(input_shape[1], unit), 
                            #Initialisation aleatoire des poids suivant une distribution
                            #de type uniforme
                            initializer="uniform",
                            #Ces poids la peuvent etre modifies lors de la descente de 
                            #gradient et de l'entrainement du model
                            trainable=True
                            )
            self.weights_list.append(weights) #Ajout des poids a notre liste
            input_shape = (None, unit) #
            layerNb += 1
        super(CustomLayer, self).build(input_shape) #Build de la class parente
        
    #La methode call() permet de set up la valeur d'acivation de nos couches
    def call(self, x):
        output = x
        for weights, activation in zip(self.weights_list, self.activations):
            print(weights, activation)
            #Chacun des neurones d'un layer va etre multiplier par les poids
            output = tf.matmul(output, weights)
            #Application des fonctions d'activations sur l'output de la couche
            if activation == "relu":
                output = tf.nn.relu(output)
            elif activation == "sigmoid":
                output = tf.nn.sigmoid(output)
            elif activation == "softmax":
                output = tf.nn.softmax(output)
        return output
        

# --> Ajout de la couche personnalisee au model

In [56]:
model = tf.keras.models.Sequential()
#2 layers, une de 4 et une de 2 neurones. Activation relu puis softmax.
#Le __init__() est trigger dans le add()
#On ne sait pas encore quel input on va recevoir dans ces couches.
model.add(tf.keras.layers.Dense(5, activation="relu")) #On peut ajouter des couches non personnalisee
model.add(CustomLayer([4, 2], ["relu", "softmax"])) 
prediction = model.predict(np.zeros((5, 10))) #Notre methode build() est appele lors de la prediction
print("Predictions :\n", prediction)

Unit :  4 , Input_shape :  (None, 5)
Unit :  2 , Input_shape :  (None, 4)
<tf.Variable 'custom_layer_17/weight-0:0' shape=(5, 4) dtype=float32> relu
<tf.Variable 'custom_layer_17/weight-1:0' shape=(4, 2) dtype=float32> softmax
<tf.Variable 'custom_layer_17/weight-0:0' shape=(5, 4) dtype=float32> relu
<tf.Variable 'custom_layer_17/weight-1:0' shape=(4, 2) dtype=float32> softmax
<tf.Variable 'custom_layer_17/weight-0:0' shape=(5, 4) dtype=float32> relu
<tf.Variable 'custom_layer_17/weight-1:0' shape=(4, 2) dtype=float32> softmax
Predictions :
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
